In [15]:
import torch

# torch 求导

参考 [url](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html)

pytorch 实现模型训练需要完整地写下训练过程，包括反向传播求梯度以及应用梯度下降算法。（06见chapter_2/03_...)

## 近似求导

In [16]:
def f(x):
    return 3. * x ** 2 + 2. * x - 1
#近视求导，x移动eps单位，也就是离自己很近的一个点的切线
def approximate_derivative(f, x, eps=1e-6):
    return (f(x + eps) - f(x - eps)) / (2. * eps)

print(approximate_derivative(f, 1.))

7.999999999785956


In [17]:
#求偏导数,其中一个数不动，对另外一个变量求导
def g(x1, x2):
    return (x1 + 5) * (x2 ** 2)

def approximate_gradient(g, x1, x2, eps=1e-3):
    dg_x1 = approximate_derivative(lambda x: g(x, x2), x1, eps)
    dg_x2 = approximate_derivative(lambda x: g(x1, x), x2, eps)
    return dg_x1, dg_x2

print(approximate_gradient(g, 2., 3.))
    

(8.999999999993236, 41.999999999994486)


## torch 近似求导

In [18]:
# 声明两个tensor x1 和 x2，允许梯度计算，使用torch的自动求导上下文计算两个tensor的梯度
# 使用 torch.autograd.grad 计算 y = g(x1, x2) 的偏导数

x1 = torch.tensor([2.], requires_grad=True)
x2 = torch.tensor([3.], requires_grad=True)
y = g(x1, x2)
    
(dy_dx1,) = torch.autograd.grad(y, x1,retain_graph=True)
print(dy_dx1)


tensor([9.])


In [19]:
try: #不加retain_graph=True第二次执行会报错，原因是因为计算图已经被释放了
    (dy_dx2,) = torch.autograd.grad(y, x2,retain_graph=True)
    print(dy_dx2)
except Exception as e:
    print(e)

tensor([42.])


In [20]:
# 同时求导

x1 = torch.tensor([2.], requires_grad=True)
x2 = torch.tensor([3.], requires_grad=True)
y = g(x1, x2)

# 求偏导数
dy_dx1, dy_dx2 = torch.autograd.grad(y, [x1, x2])


print(dy_dx1, dy_dx2)

tensor([9.]) tensor([42.])


In [21]:
# 当然我们一般直接用 backward

x1 = torch.tensor([2.], requires_grad=True)
x2 = torch.tensor([3.], requires_grad=True)
y = g(x1, x2)

# 求偏导数,求梯度
y.backward()
print(x1.grad, x2.grad)

tensor([9.]) tensor([42.])


## 二阶导


In [22]:
x1 = torch.tensor([2.], requires_grad=True)
x2 = torch.tensor([3.], requires_grad=True)
y = g(x1, x2)

# 求y对x1和x2的二阶偏导数
#，allow_unused 参数的作用是控制当 inputs 中的某些张量不需要梯度时，函数的行为方式。
# allow_unused=True 这个参数的作用是 允许计算图中某些张量没有被使用，即在计算梯度时允许部分张量的梯度不被计算或为 None。
dy_dx1, dy_dx2 = torch.autograd.grad(y, [x1, x2], create_graph=True)
dy_dx1_dx1, dy_dx1_dx2 = torch.autograd.grad(dy_dx1, [x1, x2], allow_unused=True)
dy_dx2_dx1, dy_dx2_dx2 = torch.autograd.grad(dy_dx2, [x1, x2], allow_unused=True)
print(dy_dx1_dx1, dy_dx2_dx1, dy_dx2_dx2)

None tensor([6.]) tensor([14.])


In [23]:
#模拟梯度下降算法 SGD
import torch
learning_rate = 0.3 # 每次参数更新时的步长
x = torch.tensor(2.0, requires_grad=True)
for _ in range(100):# 下划线 _ 是一个惯用法，表示 占位符，即它用于指代一个不需要用到的变量。range(100) 会返回 0 到 99 的数字。如果我们不关心这些数字的具体值（即不需要用到它们），就可以使用 _ 来代替变量名。
    z = f(x)
    z.backward()    # 计算 z 对所有涉及的张量的梯度。在本例中，z 是通过 x 计算得到的，所以 x 的梯度会被计算出来，并存储在 x.grad 中。
    x.data.sub_(learning_rate * x.grad) # x -= learning_rate * x.grad，这里就等价于optimizer.step()
    x.grad.zero_() # x.grad -= x.grad, x.grad = 0,梯度清零
    # 由于 PyTorch 会自动累积梯度（即每次调用 backward() 时，梯度会被加到现有的梯度上），所以在每次更新参数后，我们需要 手动清零梯度，以便下次计算时不会受到前次梯度的影响。
print(x)

tensor(-0.3333, requires_grad=True)


In [24]:
a=torch.tensor(2) # 标量
a.shape

torch.Size([])

In [25]:
#GradientTape与optimizer（优化器）结合使用
learning_rate = 0.01
x = torch.tensor(2.0, requires_grad=True)
optimizer = torch.optim.SGD([x], lr=learning_rate,momentum=0.9)
for _ in range(500):
    z = f(x)
    optimizer.zero_grad() # 梯度变为0
    z.backward() # dz/dx,求梯度
    # print(x.grad)
    optimizer.step() # x -= learning_rate * x.grad
    

print(x)


tensor(-0.3333, requires_grad=True)
